In [1]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import librosa
from sklearn.pipeline import make_pipeline
import os
from moviepy.editor import VideoFileClip
import cv2 as cv
from textblob import TextBlob
import whisper
from fer import FER 
from pydub import AudioSegment
import joblib
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


[nltk_data] Downloading package vader_lexicon to C:\Users\Kalash
[nltk_data]     Srivastava\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
data = {
    "Text":[
           "I'm so excited about this amazing opportunity! This is the best day ever!",
        "Just got promoted at work! Life is absolutely wonderful right now!",
        "Had the most incredible vacation with my family. Pure joy and happiness!",
        "Feeling fantastic today! Everything is going perfectly!",
        "I'm overjoyed with this wonderful news! Best feeling ever!",
        "Such a beautiful day! I'm filled with happiness and gratitude!",
        "Amazing results! I'm thrilled beyond words!",

        
 
        "I feel so lonely and empty inside. Nothing seems to matter anymore.",
        "Lost my best friend today. The sadness is overwhelming and unbearable.",
        "Everything is falling apart. I can't stop crying and feeling hopeless.",
        "Heartbroken and devastated. Life feels meaningless right now.",
        "Feeling so down and depressed. Nothing brings me joy anymore.",
        "The grief is consuming me. I feel so lost and alone.",
        "Tears won't stop flowing. This pain is unbearable.",

        
      
        "I'm absolutely furious! This incompetent service is completely unacceptable!",
        "How dare they treat me like this! I'm fed up with this ridiculous situation!",
        "This makes my blood boil! I'm sick and tired of being ignored and disrespected!",
        "I'm outraged by this injustice! This is completely infuriating!",
        "Absolutely livid! This is the most frustrating experience ever!",
        "I'm seething with anger! This is beyond acceptable!",
        "Rage is building up inside me! This is completely outrageous!",
    
        
 
        "I'm terrified about the upcoming surgery. What if something goes wrong?",
        "Walking alone at night makes me anxious. I'm scared something bad will happen.",
        "The thought of public speaking fills me with dread and panic.",
        "I'm petrified of what might happen next. This uncertainty is killing me.",
        "Anxiety is overwhelming me. I'm scared of the unknown.",
        "Trembling with fear about the future. Everything seems so uncertain.",
        "I'm frightened by these strange noises. Something feels very wrong.",

   
        "That food was absolutely revolting. I almost threw up from the smell.",
        "This place is filthy and repulsive. I can't stand being here anymore.",
        "The way they handled this situation is disgusting and makes me sick.",
        "This is absolutely nauseating. I feel physically ill from this.",
        "Gross! This is the most disgusting thing I've ever seen!",
        "I'm repulsed by this behavior. It's completely sickening.",
        "This smell is making me gag. Absolutely revolting!",
     
        
       
        "The weather today is partly cloudy with a chance of rain.",
        "I went to the store and bought some groceries for dinner.",
        "The meeting is scheduled for 3 PM in the conference room.",
        "Today is Monday and the temperature is 72 degrees.",
        "I need to finish this report by the end of the week.",
        "The train arrives at platform 3 at 2:30 PM.",
        "Please remember to bring your ID card to the office.",
   
        ],

    
    "Video_path":[
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy2.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy3.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy4.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy5.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy6.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\happy7.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad2.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad3.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad4.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad5.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad6.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\sad7.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry2.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry3.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry4.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry5.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry6.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry7.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear2.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear3.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear4.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear5.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear6.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\fear7.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust2.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust3.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust4.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust5.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust6.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\disgust7.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral2.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral3.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral4.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral5.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral6.mp4",
                  r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\neutral7.mp4",
              

                  ],
    
    
    
    "Emotions":[  
         "happy", "happy", "happy", "happy", "happy", "happy", "happy",
        
        "sad", "sad", "sad", "sad", "sad", "sad", "sad", 
    
        "angry", "angry", "angry", "angry", "angry", "angry", "angry", 
    
        "fear", "fear", "fear", "fear", "fear", "fear", "fear", 
     
        "disgust", "disgust", "disgust", "disgust", "disgust", "disgust", "disgust",
   
        "neutral", "neutral", "neutral", "neutral", "neutral", "neutral", "neutral",
    ]
    
    
}

In [3]:
data = pd.DataFrame(data)
data.to_csv("data.csv",index = False)

In [4]:
stem = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    words = word_tokenize(text)
    filtered = [stem.stem(word)for word in words if word not in stop_words]
    return ' '.join(filtered)
    
data["cleaned_text"] = data["Text"].apply(clean_text)

In [6]:
data["Emotions"].value_counts()

Emotions
happy      7
sad        7
angry      7
fear       7
disgust    7
neutral    7
Name: count, dtype: int64

In [7]:
data["Emotions"] = data["Emotions"].map({'happy':0,'sad':1,'angry':2,'fear':3,'disgust':4,'neutral':5})

In [8]:
data.head()

,Text,Video_path,Emotions,cleaned_text
0,I'm so excited about this amazing opportunity!...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,excit amaz opportun best day ever
1,Just got promoted at work! Life is absolutely ...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,got promot work life absolut wonder right
2,Had the most incredible vacation with my famil...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,incred vacat famili pure joy happi
3,Feeling fantastic today! Everything is going p...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,feel fantast today everyth go perfectli
4,I'm overjoyed with this wonderful news! Best f...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,overjoy wonder news best feel ever


In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data["cleaned_text"])
y = data["Emotions"]

In [10]:
vectorizer.get_feature_names_out()

array(['absolut', 'accept', 'almost', 'alon', 'amaz', 'anger', 'anxieti',
       'anxiou', 'anymor', 'apart', 'arriv', 'bad', 'beauti', 'behavior',
       'best', 'beyond', 'blood', 'boil', 'bought', 'bring', 'build',
       'card', 'chanc', 'cloudi', 'complet', 'confer', 'consum', 'cri',
       'dare', 'day', 'degre', 'depress', 'devast', 'dinner', 'disgust',
       'disrespect', 'dread', 'empti', 'end', 'ever', 'everyth', 'excit',
       'experi', 'fall', 'famili', 'fantast', 'fear', 'fed', 'feel',
       'fill', 'filthi', 'finish', 'flow', 'food', 'friend', 'frighten',
       'frustrat', 'furiou', 'futur', 'gag', 'go', 'goe', 'got',
       'gratitud', 'grief', 'groceri', 'gross', 'handl', 'happen',
       'happi', 'heartbroken', 'hopeless', 'id', 'ignor', 'ill',
       'incompet', 'incred', 'infuri', 'injustic', 'insid', 'joy', 'kill',
       'life', 'like', 'livid', 'lone', 'lost', 'make', 'matter',
       'meaningless', 'meet', 'might', 'monday', 'nauseat', 'need',
       'news', 

In [11]:
X_dense = X.toarray()
print(X_dense)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.28465361 0.         0.         ... 0.         0.41132031 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [12]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity >0.1:
        return "possitive"
    elif polarity <-0.1:
        return "negative"
    else:
        return"neutral"
data["text_sentiment"] = data["Text"].apply(analyze_sentiment)


In [13]:
data[["Emotions","text_sentiment"]]

,Emotions,text_sentiment
0,0,possitive
1,0,possitive
2,0,possitive
3,0,possitive
4,0,possitive
5,0,possitive
6,0,possitive
7,1,neutral
8,1,possitive
9,1,negative


In [14]:
audio_folder = "Face Emotions"
emotions = ["happy","sad","angry","fear","disgust","neutral"]

features = []
labels = []

files = os.listdir(audio_folder)
for file in files:
    if file.endswith(".mp4"):
        for emotion in emotions:
            if emotion in file.lower():
                try:
                    file_path = os.path.join(audio_folder,file)
                    y,sr =librosa.load(file_path,sr=None)
                    mfcc = librosa.feature.mfcc(y=y,sr=sr,n_mfcc=13)
                    mfcc_mean = np.mean(mfcc.T,axis=0)
                    features.append(mfcc_mean)
                    labels.append(emotion)
                    print(f"Processed: {file} as {emotion}")
                except Exception as e:
                    print(f" Error processing {file}: {e}")
                    break

if len(features) == 0:
    print("No feature were extracted ")
else:

    X = np.array(features)
    y = np.array(labels)

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    model = RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    print(classification_report(y_test,y_pred))
    joblib.dump(model,"audio_emotion_model.pkl")
    print("Model saved successfully")


  y,sr =librosa.load(file_path,sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

  y,sr =librosa.load(file_path,sr=None)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



Processed: angry.mp4 as angry
Processed: angry2.mp4 as angry
Processed: angry3.mp4 as angry
Processed: angry5.mp4 as angry
Processed: angry6.mp4 as angry
Processed: angry7.mp4 as angry
Processed: disgust.mp4 as disgust
Processed: disgust2.mp4 as disgust
Processed: disgust3.mp4 as disgust
Processed: disgust4.mp4 as disgust
Processed: disgust5.mp4 as disgust
Processed: disgust6.mp4 as disgust
Processed: disgust7.mp4 as disgust
Processed: fear.mp4 as fear
Processed: fear2.mp4 as fear
Processed: fear3.mp4 as fear
Processed: fear4.mp4 as fear
Processed: fear5.mp4 as fear
Processed: fear6.mp4 as fear
Processed: fear7.mp4 as fear
Processed: happy.mp4 as happy
Processed: happy2.mp4 as happy
Processed: happy3.mp4 as happy
Processed: happy4.mp4 as happy
Processed: happy5.mp4 as happy
Processed: happy6.mp4 as happy
Processed: happy7.mp4 as happy
Processed: neutral.mp4 as neutral
Processed: neutral2.mp4 as neutral
Processed: neutral3.mp4 as neutral
Processed: neutral4.mp4 as neutral
Processed: neu

In [15]:
video_folder = "Face Emotions"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]
face_detector = FER(mtcnn=False)
whisper_model = whisper.load_model("base")
audio_model = joblib.load("audio_emotion_model.pkl")
result = []

for file in video_files:
    video_path = os.path.join(video_folder, file)

    cap = cv.VideoCapture(video_path)
    frame_count, frame_skip = 0, 5
    emotions_list = []


    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue

        detected = face_detector.detect_emotions(frame)
        if detected:
            emotions = detected[0]["emotions"]
            dominant = max(emotions, key=emotions.get)
            emotions_list.append(dominant)

    cap.release()


    if emotions_list:
        dominant_face_emotion = max(set(emotions_list), key=emotions_list.count)
    else:
        dominant_face_emotion = "undetected"


    audio_sentiment = "no_audio"
    transcript = ""
    temp_mp3 = "temp_audio.mp3"
    temp_wav = "temp_audio.wav"

    try:
        video_clip = VideoFileClip(video_path)
        if video_clip.audio is None:
            print(" No audio in found")
        else:
   
            video_clip.audio.write_audiofile(temp_mp3, verbose=False, logger=None)
            sound = AudioSegment.from_file(temp_mp3)
            sound.export(temp_wav, format="wav")

            if os.path.exists(temp_wav):

                y, sr = librosa.load(temp_wav, sr=None)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
                mfcc_processed = np.mean(mfcc.T,axis=0).reshape(1,-1)
                predicted_emotion = audio_model.predict(mfcc_processed)[0]
                audio_sentiment = predicted_emotion
                
                

    except Exception as e:
        print(f"Error in audio processing for {file}: {e}")
        audio_sentiment = "error"
        transcript = ""


    for temp_file in [temp_mp3, temp_wav]:
        if os.path.exists(temp_file):
            os.remove(temp_file)


    result.append({
        "video_file": file,
        "face_emotion": dominant_face_emotion,
        "audio_sentiment": audio_sentiment,
        "transcribed_text": transcript,
        "emotion_list":emotions_list
    })


df = pd.DataFrame(result)
df.to_csv("Face_emotion_detection.csv", index=False)
print(" Successfully saved to Face_emotion_detection.csv")


 Successfully saved to Face_emotion_detection.csv


In [16]:
df.head()

,video_file,face_emotion,audio_sentiment,transcribed_text,emotion_list
0,angry.mp4,neutral,angry,,"[neutral, neutral, neutral, neutral, neutral, ..."
1,angry2.mp4,neutral,angry,,"[neutral, neutral, neutral, neutral, neutral, ..."
2,angry3.mp4,neutral,fear,,"[neutral, neutral, neutral, neutral, neutral, ..."
3,angry5.mp4,neutral,angry,,"[neutral, neutral, neutral, neutral, neutral, ..."
4,angry6.mp4,neutral,fear,,"[neutral, neutral, neutral, neutral, neutral, ..."


In [17]:
data.head()

,Text,Video_path,Emotions,cleaned_text,text_sentiment
0,I'm so excited about this amazing opportunity!...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,excit amaz opportun best day ever,possitive
1,Just got promoted at work! Life is absolutely ...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,got promot work life absolut wonder right,possitive
2,Had the most incredible vacation with my famil...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,incred vacat famili pure joy happi,possitive
3,Feeling fantastic today! Everything is going p...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,feel fantast today everyth go perfectli,possitive
4,I'm overjoyed with this wonderful news! Best f...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,overjoy wonder news best feel ever,possitive


In [18]:
merged_df = pd.concat([data, df], axis=1)

In [20]:
merged_df.head()

,Text,Video_path,Emotions,cleaned_text,text_sentiment,video_file,face_emotion,audio_sentiment,transcribed_text,emotion_list
0,I'm so excited about this amazing opportunity!...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,excit amaz opportun best day ever,possitive,angry.mp4,neutral,angry,,"[neutral, neutral, neutral, neutral, neutral, ..."
1,Just got promoted at work! Life is absolutely ...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,got promot work life absolut wonder right,possitive,angry2.mp4,neutral,angry,,"[neutral, neutral, neutral, neutral, neutral, ..."
2,Had the most incredible vacation with my famil...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,incred vacat famili pure joy happi,possitive,angry3.mp4,neutral,fear,,"[neutral, neutral, neutral, neutral, neutral, ..."
3,Feeling fantastic today! Everything is going p...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,feel fantast today everyth go perfectli,possitive,angry5.mp4,neutral,angry,,"[neutral, neutral, neutral, neutral, neutral, ..."
4,I'm overjoyed with this wonderful news! Best f...,C:\Users\Kalash Srivastava\Demo-Projects\Multi...,0,overjoy wonder news best feel ever,possitive,angry6.mp4,neutral,fear,,"[neutral, neutral, neutral, neutral, neutral, ..."


In [21]:
merged_df[["text_sentiment","audio_sentiment","face_emotion"]].value_counts().unique

<bound method Series.unique of text_sentiment  audio_sentiment  face_emotion
negative        fear             surprise        4
neutral         disgust          sad             4
possitive       angry            neutral         4
                fear             neutral         3
negative        neutral          neutral         3
neutral         happy            happy           3
possitive       neutral          neutral         2
                disgust          sad             2
negative        happy            happy           2
                neutral          happy           2
neutral         sad              sad             2
                                 fear            1
possitive       neutral          happy           1
neutral         sad              neutral         1
negative        disgust          neutral         1
neutral         neutral          neutral         1
negative        disgust          sad             1
neutral         fear             fear            1
negat

In [22]:
merged_df["text_sentiment_encoded"] = merged_df["text_sentiment"].map({"possitive":0,"negative":1,"neutral":2})
merged_df["audio_sentiment_encoded"] = merged_df["audio_sentiment"].map({"happy":0,"sad":1,"angry":2,"fear":3,"disgust":4,"neutral":5})
merged_df["face_emotion_encoded"] = merged_df["face_emotion"].map({"happy":0,"sad":1,"fear":2,"suprise":3,"neutral":4})


In [23]:
from moviepy.editor import VideoFileClip

video_path = r"C:\Users\Kalash Srivastava\Demo-Projects\Multi-Modal Sentiment Analysis'\Face Emotions\angry.mp4"
video = VideoFileClip(video_path)

# Check if audio exists
if video.audio is None:
    print("No audio detected")
else:
    print("Audio is detected")
    video.audio.write_audiofile("test_extracted.wav", verbose=False)
    print("Saved to test_extracted.wav")


Audio is detected
MoviePy - Writing audio in test_extracted.wav


MoviePy - Done.
Saved to test_extracted.wav


In [25]:

other_feature = merged_df[["text_sentiment_encoded", "audio_sentiment_encoded", "face_emotion_encoded"]].values

X_final = np.concatenate([X_dense,other_feature],axis=1)

y = merged_df["Emotions"]

X_train,X_test,y_train,y_test = train_test_split(X_final,y,test_size=0.2,random_state=42)


model = RandomForestClassifier()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

report = classification_report(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)

print("The Accuracy is :",accuracy)
print("The classification report is :",report)

  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



The Accuracy is : 0.5555555555555556
The classification report is :               precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         2
           2       1.00      1.00      1.00         1
           3       1.00      0.50      0.67         2
           4       0.50      1.00      0.67         2
           5       0.00      0.00      0.00         1

    accuracy                           0.56         9
   macro avg       0.50      0.58      0.50         9
weighted avg       0.50      0.56      0.48         9

